# Data collection

Here I am going to collect all the necessary data from `html` files using my package called [`lattes_html_parser`](https://github.com/guilhermecxe/lattes-html-parser). At the end, I saved a pandas dataframe with these columns:

    - researcher_name
    - researcher_lattes_id
    - researcher_major_field
    - article_title.

In [6]:
from lattes_html_parser import get_researchers
# Know more about lattes_html_parser: https://github.com/guilhermecxe/lattes-html-parser

import pandas as pd
import pickle
import os

In [4]:
DATA_PATH = 'data/researchers-by-area'

In [5]:
%%time

# Creating instances of researchers from html files using the lattes_html_parser package
researchers = []
for folder in os.listdir(DATA_PATH):
    area_path = os.path.join(DATA_PATH, folder)
    researchers += get_researchers(area_path)

print('Researchers obtained:', len(researchers))

Researchers obtained: 2000
CPU times: total: 2min 40s
Wall time: 3min 57s


In [14]:
# Example of how the researcher area of expertise is represented
researchers[54].areas_of_expertise

['Ciências Agrárias > Agronomia > Ciência do Solo > Física do Solo.',
 'Ciências Agrárias > Agronomia > Ciência do Solo > Operações Agrícolas.']

In [15]:
# Function to get researcher most frequent major field
def major_field(researcher):
    major_fields = [area.split(' > ')[0] for area in researcher.areas_of_expertise]
    major_field = max(major_fields, key=major_fields.count)
    return major_field.strip('.')

# Example
print("Researcher areas of expertise:")
print(researchers[45].areas_of_expertise)
print()
print("Researcher most frequent major field:")
print(major_field(researchers[45]))

Researcher areas of expertise:
['Ciências Agrárias > Agronomia > Ciência do Solo.', 'Ciências Agrárias > Agronomia > Ciência do Solo > Gênese, Morfologia e Classificação dos Solos.', 'Ciências Agrárias > Agronomia > Ciência do Solo > Manejo e Conservação do Solo.', 'Ciências Agrárias > Agronomia > Ciência do Solo > Levantamento de Solos.', 'Ciências Agrárias > Agronomia > Ciência do Solo > Mineralogia dos Solos.', 'Ciências Agrárias > Agronomia > Ciência do Solo > Zoneamento Agroecológico.']

Researcher most frequent major field:
Ciências Agrárias


In [16]:
columns = ['researcher_name', 'researcher_lattes_id', 'researcher_major_field', 'article_title']
rows = []

# Collecting researchers and their articles
for researcher in researchers:
    name = researcher.name
    lattes_id = researcher.lattes_id
    researcher_major_field = major_field(researcher)
    for article in researcher.complete_articles:
        title = article.title
        if title: # Some articles don't have title
            rows.append({
                'researcher_name': name,
                'researcher_lattes_id': lattes_id,
                'researcher_major_field': researcher_major_field,
                'article_title': title
            })

In [17]:
# Making the data collected a dataframe
articles_and_fields_df = pd.DataFrame(rows, columns=columns)

print('Dataframe shape:', articles_and_fields_df.shape)
articles_and_fields_df.head()

Dataframe shape: (14735, 4)


,researcher_name,researcher_lattes_id,researcher_major_field,article_title
0,Juarez Morbini Lopes,2612407908632983,Ciências Agrárias,Níveis das vitaminas A e E em dietas de frango...
1,Juarez Morbini Lopes,2612407908632983,Ciências Agrárias,Adição de bentonita sódica como adsorvente de ...
2,Juarez Morbini Lopes,2612407908632983,Ciências Agrárias,Níveis de substituição da DL-metionina pela me...
3,Juarez Morbini Lopes,2612407908632983,Ciências Agrárias,Enzimas de função hepática na aflatoxicose agu...
4,Juarez Morbini Lopes,2612407908632983,Ciências Agrárias,"Efeitos de Níveis das VitaminasA, E, Piridoxin..."


In [18]:
# Saving it as pickle for later use
with open('data/articles_and_fields_df.pickle', 'wb') as f:
    pickle.dump(articles_and_fields_df, f)